In [1]:
import argparse
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import datetime
import time
import logging
from joblib import dump, load
from datetime import timedelta
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import RegressorChain
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
import xgboost as xgb
import lightgbm as lightgbm

# settings:
numberOfInputWeeks = 3 # must be equal to the number of input weeks set in data preperator
numberOfOutputWeeks = 4 # must be equal to the number of output week set in data preperator



# data preperation
cantonKeys = ['AG','AI','AR', 'BE', 'BL', 'BS', 'FR', 'GE', 'GL', 'GR', 'JU', 'LU', 'NE', 'NW', 'OW', 'SG', 'SH', 'SO', 'SZ', 'TG', 'TI', 'UR', 'VD', 'VS', 'ZG','ZH']
data = pd.read_csv("completedata.csv")


outputCategories = ['case_inz_entries_7dayAverage',
                  'hosp_inz_entries_7dayAverage',
                  'death_inz_entries_7dayAverage',
                  'testPositvity_7dayAverage',
                  'transit_stations_percent_change_from_baseline_7dayAverage',
                  'workplaces_percent_change_from_baseline_7dayAverage'
                   ]
numberOfOutputs = len(outputCategories)

split = numberOfOutputs * numberOfOutputWeeks + 2
train_features = data[data['category']=='train'].iloc[:,0:-split].drop(['lastInputDay'], axis=1)
train_labels = data[data['category']=='train'].iloc[:,-split:-2]

validation1_features = data[data['category']=='validation 1'].iloc[:,0:-split].drop(['lastInputDay'], axis=1)
validation1_labels = data[data['category']=='validation 1'].iloc[:,-split:-2]
validation2_features = data[data['category']=='validation 2'].iloc[:,0:-split].drop(['lastInputDay'], axis=1)
validation2_labels = data[data['category']=='validation 2'].iloc[:,-split:-2]
validation1And2_labels = data[(data['category']=='validation 1') | (data['category']=='validation 2')].iloc[:,-split:-2]
validation1And2_features = data[(data['category']=='validation 1') | (data['category']=='validation 2')].iloc[:,0:-split].drop(['lastInputDay'], axis=1)

pip = Pipeline([('minmax_scaler', MinMaxScaler())])
X_train = pip.fit_transform(train_features[train_features.columns].values)
X_valid1 = pip.transform(validation1_features[train_features.columns].values)
X_valid2 = pip.transform(validation2_features[train_features.columns].values)
X_valid1And2 = pip.transform(validation1And2_features[train_features.columns].values)


def generic1(learningrate, hiddenLayers, dropout, l1, l2, isMultiOutput):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(hiddenLayers[0], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2), input_shape=X_train.shape[1:]))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(numberOfOutputWeeks) if isMultiOutput else keras.layers.Dense(1))
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model
    
def generic2(learningrate, hiddenLayers, dropout, l1, l2, isMultiOutput):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(hiddenLayers[0], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2), input_shape=X_train.shape[1:]))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[1], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(numberOfOutputWeeks) if isMultiOutput else keras.layers.Dense(1))
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def generic3(learningrate, hiddenLayers, dropout, l1, l2, isMultiOutput):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(hiddenLayers[0], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2), input_shape=X_train.shape[1:]))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[1], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[2], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(numberOfOutputWeeks) if isMultiOutput else keras.layers.Dense(1))
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def generic4(learningrate, hiddenLayers, dropout, l1, l2, isMultiOutput):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(hiddenLayers[0], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2), input_shape=X_train.shape[1:]))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[1], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[2], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[3], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(numberOfOutputWeeks) if isMultiOutput else keras.layers.Dense(1))
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def generic5(learningrate, hiddenLayers, dropout, l1, l2, isMultiOutput):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(hiddenLayers[0], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2), input_shape=X_train.shape[1:]))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[1], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[2], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[3], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[4], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(numberOfOutputWeeks) if isMultiOutput else keras.layers.Dense(1))
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def generic6(learningrate, hiddenLayers, dropout, l1, l2, isMultiOutput):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(hiddenLayers[0], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2), input_shape=X_train.shape[1:]))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[1], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[2], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[3], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[4], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[5], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(numberOfOutputWeeks) if isMultiOutput else keras.layers.Dense(1))
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def generic7(learningrate, hiddenLayers, dropout, l1, l2, isMultiOutput):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(hiddenLayers[0], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2), input_shape=X_train.shape[1:]))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[1], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[2], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[3], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[4], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[5], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[6], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(numberOfOutputWeeks) if isMultiOutput else keras.layers.Dense(1))
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model

def generic8(learningrate, hiddenLayers, dropout, l1, l2, isMultiOutput):
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(hiddenLayers[0], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2), input_shape=X_train.shape[1:]))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[1], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[2], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[3], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[4], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[5], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[6], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(hiddenLayers[7], activation="relu", kernel_regularizer=keras.regularizers.l1_l2(l1=l1, l2=l2)))
    model.add(keras.layers.Dropout(rate=dropout))
    model.add(keras.layers.Dense(numberOfOutputWeeks) if isMultiOutput else keras.layers.Dense(1))
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.Adam(learning_rate=learningrate))
    return model
    

    
def genericKerasModel(learningrate, hiddenLayers, dropout, l1regularization, l2regularization, isMultioutput):
    if len(hiddenLayers) == 1:
        return generic1(learningrate, hiddenLayers, dropout, l1regularization, l2regularization, isMultioutput)
    elif len(hiddenLayers) == 2:
        return generic2(learningrate, hiddenLayers, dropout, l1regularization, l2regularization, isMultioutput)
    elif len(hiddenLayers) == 3:
        return generic3(learningrate, hiddenLayers, dropout, l1regularization, l2regularization, isMultioutput)
    elif len(hiddenLayers) == 4:
        return generic4(learningrate, hiddenLayers, dropout, l1regularization, l2regularization, isMultioutput)
    elif len(hiddenLayers) == 5:
        return generic5(learningrate, hiddenLayers, dropout, l1regularization, l2regularization, isMultioutput)
    elif len(hiddenLayers) == 6:
        return generic6(learningrate, hiddenLayers, dropout, l1regularization, l2regularization, isMultioutput)
    elif len(hiddenLayers) == 7:
        return generic7(learningrate, hiddenLayers, dropout, l1regularization, l2regularization, isMultioutput)
    else:
        return generic8(learningrate, hiddenLayers, dropout, l1regularization, l2regularization, isMultioutput)
    

parser = argparse.ArgumentParser()
parser.add_argument("selectedEstimator", help="computes estimator with given id",type=int)
args = parser.parse_args()
selectedEstimator = args.selectedEstimator
'''
selectedEstimator = 6
'''

topComplete = pd.DataFrame()
for task in outputCategories:
    temp = pd.read_csv("top100/"+task+".csv")
    temp['task'] = task
    topComplete = topComplete.append(temp)



In [2]:
estimators = []
numberOfEstimators = 0
results = pd.DataFrame()


estimatorOfInterest = topComplete.iloc[selectedEstimator]

estimatorId = 0
#df['Tags'] = df.Tags.apply(lambda x: x[1:-1].split(','))
hl = estimatorOfInterest['hiddenLayers'][1:-1].split(',')
hl = [int(x) for x in hl]



estimators.append(
                        {
                          "modelClass": "Keras",
                          "isMultiWeek": True,
                          "hiddenLayers": hl,
                          "numberOfhiddenLayers": len(hl),
                          "isTwoWay": False,
                          "l1reg": estimatorOfInterest['l1reg'],
                          "alpha": estimatorOfInterest['alpha'],
                          "dropoutValue": estimatorOfInterest['dropoutValue'],
                          "learningRate": estimatorOfInterest['learningRate'],
                          "task": estimatorOfInterest['task']
                        }
                    )



# constructs and saves the results of a fitted estimator    
def constructResults(estimator, task, weekNumber, numberOfRanEpochs):
    if weekNumber == -1: # we make predictions for all weeks
        # predictions for all weeks
        predictions1 = pd.DataFrame(estimator.predict(X_valid1), index=validation1_labels.index, columns=["pred_week_"+task+"_"+str(outputWeekNumber) for outputWeekNumber in range(0,numberOfOutputWeeks)]) 
        predictions2 = pd.DataFrame(estimator.predict(X_valid2), index=validation2_labels.index, columns=["pred_week_"+task+"_"+str(outputWeekNumber) for outputWeekNumber in range(0,numberOfOutputWeeks)])
        
        # validation for all weeks
        y_valid1 = validation1_labels[["output_"+task+"_"+str(outputWeekNumber) for outputWeekNumber in range(0,numberOfOutputWeeks)]]
        y_valid2 = validation2_labels[["output_"+task+"_"+str(outputWeekNumber) for outputWeekNumber in range(0,numberOfOutputWeeks)]]
        
        resultsDf = pd.DataFrame()
        # compute and safe results for every week
        for outputWeekNumber in range(0,numberOfOutputWeeks):
            # compute root mean squared error for validation sets
            rmse1 = np.sqrt(mean_squared_error(predictions1["pred_week_"+task+"_"+str(outputWeekNumber)], y_valid1["output_"+task+"_"+str(outputWeekNumber)]))
            rmse2 = np.sqrt(mean_squared_error(predictions2["pred_week_"+task+"_"+str(outputWeekNumber)], y_valid2["output_"+task+"_"+str(outputWeekNumber)]))
            # safe the results and all model parameters
            res = {}
            if estimators[estimatorId]["modelClass"] == "Keras":
                res = {'modelClass':[estimators[estimatorId]["modelClass"]],
                     'task':[task],
                     'week':[outputWeekNumber], 
                     'model rmse 1':[rmse1], 
                     'model rmse 2':[rmse2],
                     'isMultiWeek':[estimators[estimatorId]["isMultiWeek"]],
                     "hiddenLayers":[estimators[estimatorId]["hiddenLayers"]],
                     "numberOfhiddenLayers":[len(estimators[estimatorId]["hiddenLayers"])],
                     "isTwoWay": [estimators[estimatorId]["isTwoWay"]],
                     "l1reg": [estimators[estimatorId]["l1reg"]],
                     "alpha": [estimators[estimatorId]["alpha"]],
                     "dropoutValue":[estimators[estimatorId]["dropoutValue"]],
                     "learningRate":[estimators[estimatorId]["learningRate"]],
                     "numberOfRanEpochs": [numberOfRanEpochs]
                    }
                
            elif estimators[estimatorId]["modelClass"] == "RandomForrest":
                res = {'modelClass':[estimators[estimatorId]["modelClass"]],
                     'task':[task],
                     'week':[outputWeekNumber], 
                     'model rmse 1':[rmse1], 
                     'model rmse 2':[rmse2],
                     'isMultiWeek':[estimators[estimatorId]["isMultiWeek"]],
                     "n_estimators": [estimators[estimatorId]["n_estimators"]],
                     "max_features": [estimators[estimatorId]["max_features"]],
                     "min_samples_split": [estimators[estimatorId]["min_samples_split"]],
                     "min_samples_leaf": [estimators[estimatorId]["min_samples_leaf"]]
                    }               
            elif estimators[estimatorId]["modelClass"] == "KernelRidge":
                res = {'modelClass':[estimators[estimatorId]["modelClass"]],
                     'task':[task],
                     'week':[outputWeekNumber], 
                     'model rmse 1':[rmse1], 
                     'model rmse 2':[rmse2],
                     'isMultiWeek':[estimators[estimatorId]["isMultiWeek"]],
                     "alpha": [estimators[estimatorId]["alpha"]],
                     "kernel": [estimators[estimatorId]["kernel"]],
                    }
            elif estimators[estimatorId]["modelClass"] == "Ridge":
                res = {'modelClass':[estimators[estimatorId]["modelClass"]],
                     'task':[task],
                     'week':[outputWeekNumber], 
                     'model rmse 1':[rmse1], 
                     'model rmse 2':[rmse2],
                     'isMultiWeek':[estimators[estimatorId]["isMultiWeek"]],
                     "alpha": [estimators[estimatorId]["alpha"]]
                    }
            elif estimators[estimatorId]["modelClass"] == "Lasso":
                res = {'modelClass':[estimators[estimatorId]["modelClass"]],
                     'task':[task],
                     'week':[outputWeekNumber], 
                     'model rmse 1':[rmse1], 
                     'model rmse 2':[rmse2],
                     'isMultiWeek':[estimators[estimatorId]["isMultiWeek"]],
                     "alpha": [estimators[estimatorId]["alpha"]]
                    }
            elif estimators[estimatorId]["modelClass"] == "MultiTaskLasso":
                res = {'modelClass':[estimators[estimatorId]["modelClass"]],
                     'task':[task],
                     'week':[outputWeekNumber], 
                     'model rmse 1':[rmse1], 
                     'model rmse 2':[rmse2],
                     'isMultiWeek':[estimators[estimatorId]["isMultiWeek"]],
                     "alpha": [estimators[estimatorId]["alpha"]]
                    }
            else:
                raise ValueError('Tried to save results for an unsupported estimator')
            resultsDf = resultsDf.append(pd.DataFrame(data=res), ignore_index = True)
        return resultsDf
    else: # we make predictions only for one week
        # predictions for one week
        predictions1 = pd.DataFrame(estimator.predict(X_valid1), index=validation1_labels.index, columns=["pred_week_"+task+"_"+str(weekNumber)]) 
        predictions2 = pd.DataFrame(estimator.predict(X_valid2), index=validation2_labels.index, columns=["pred_week_"+task+"_"+str(weekNumber)])
        
        # validation for one week
        y_valid1 = validation1_labels[["output_"+task+"_"+str(weekNumber)]]
        y_valid2 = validation2_labels[["output_"+task+"_"+str(weekNumber)]]
        
        rmse1 = np.sqrt(mean_squared_error(predictions1["pred_week_"+task+"_"+str(weekNumber)], y_valid1["output_"+task+"_"+str(weekNumber)]))
        rmse2 = np.sqrt(mean_squared_error(predictions2["pred_week_"+task+"_"+str(weekNumber)], y_valid2["output_"+task+"_"+str(weekNumber)]))
        # safe the results and all model parameters
        res = {}
        if estimators[estimatorId]["modelClass"] == "Keras":
            res = {'modelClass':[estimators[estimatorId]["modelClass"]],
                 'task':[task],
                 'week':[weekNumber], 
                 'model rmse 1':[rmse1], 
                 'model rmse 2':[rmse2],
                 'isMultiWeek':[estimators[estimatorId]["isMultiWeek"]],
                 "hiddenLayers":[estimators[estimatorId]["hiddenLayers"]],
                 "numberOfhiddenLayers":[len(estimators[estimatorId]["hiddenLayers"])],
                 "isTwoWay": [estimators[estimatorId]["isTwoWay"]],
                 "l1reg": [estimators[estimatorId]["l1reg"]],
                 "alpha": [estimators[estimatorId]["alpha"]],
                 "dropoutValue":[estimators[estimatorId]["dropoutValue"]],
                 "learningRate":[estimators[estimatorId]["learningRate"]],
                 "numberOfRanEpochs": [numberOfRanEpochs]
                }

        elif estimators[estimatorId]["modelClass"] == "RandomForrest":
            res = {'modelClass':[estimators[estimatorId]["modelClass"]],
                 'task':[task],
                 'week':[weekNumber], 
                 'model rmse 1':[rmse1], 
                 'model rmse 2':[rmse2],
                 'isMultiWeek':[estimators[estimatorId]["isMultiWeek"]],
                 "n_estimators": [estimators[estimatorId]["n_estimators"]],
                 "max_features": [estimators[estimatorId]["max_features"]],
                 "min_samples_split": [estimators[estimatorId]["min_samples_split"]],
                 "min_samples_leaf": [estimators[estimatorId]["min_samples_leaf"]]
                }
        elif estimators[estimatorId]["modelClass"] == "XGB":
            res = {'modelClass':[estimators[estimatorId]["modelClass"]],
                 'task':[task],
                 'week':[weekNumber], 
                 'model rmse 1':[rmse1], 
                 'model rmse 2':[rmse2],
                 'isMultiWeek':[estimators[estimatorId]["isMultiWeek"]],
                 "n_estimators": [estimators[estimatorId]["n_estimators"]],
                 "alpha": [estimators[estimatorId]["alpha"]],
                 "lambda": [estimators[estimatorId]["lambda"]]
                } 
        elif estimators[estimatorId]["modelClass"] == "LGBM":
            res = {'modelClass':[estimators[estimatorId]["modelClass"]],
                 'task':[task],
                 'week':[weekNumber], 
                 'model rmse 1':[rmse1], 
                 'model rmse 2':[rmse2],
                 'isMultiWeek':[estimators[estimatorId]["isMultiWeek"]],
                 "n_estimators": [estimators[estimatorId]["n_estimators"]],
                 "alpha": [estimators[estimatorId]["alpha"]],
                 "lambda": [estimators[estimatorId]["lambda"]]
                } 
        elif estimators[estimatorId]["modelClass"] == "KernelRidge":
            res = {'modelClass':[estimators[estimatorId]["modelClass"]],
                 'task':[task],
                 'week':[weekNumber], 
                 'model rmse 1':[rmse1], 
                 'model rmse 2':[rmse2],
                 'isMultiWeek':[estimators[estimatorId]["isMultiWeek"]],
                 "alpha": [estimators[estimatorId]["alpha"]],
                 "kernel": [estimators[estimatorId]["kernel"]],
                }
        elif estimators[estimatorId]["modelClass"] == "Ridge":
            res = {'modelClass':[estimators[estimatorId]["modelClass"]],
                 'task':[task],
                 'week':[weekNumber], 
                 'model rmse 1':[rmse1], 
                 'model rmse 2':[rmse2],
                 'isMultiWeek':[estimators[estimatorId]["isMultiWeek"]],
                 "alpha": [estimators[estimatorId]["alpha"]]
                }
        elif estimators[estimatorId]["modelClass"] == "Lasso":
            res = {'modelClass':[estimators[estimatorId]["modelClass"]],
                 'task':[task],
                 'week':[weekNumber], 
                 'model rmse 1':[rmse1], 
                 'model rmse 2':[rmse2],
                 'isMultiWeek':[estimators[estimatorId]["isMultiWeek"]],
                 "alpha": [estimators[estimatorId]["alpha"]]
                }
        elif estimators[estimatorId]["modelClass"] == "SGD":
            res = {'modelClass':[estimators[estimatorId]["modelClass"]],
                 'task':[task],
                 'week':[weekNumber], 
                 'model rmse 1':[rmse1], 
                 'model rmse 2':[rmse2],
                 'isMultiWeek':[estimators[estimatorId]["isMultiWeek"]],
                 "alpha": [estimators[estimatorId]["alpha"]],
                 "learning_rate": [estimators[estimatorId]["learning_rate"]],
                 "penalty": [estimators[estimatorId]["penalty"]]
                }
        elif estimators[estimatorId]["modelClass"] == "SVR":
            res = {'modelClass':[estimators[estimatorId]["modelClass"]],
                 'task':[task],
                 'week':[weekNumber], 
                 'model rmse 1':[rmse1], 
                 'model rmse 2':[rmse2],
                 'isMultiWeek':[estimators[estimatorId]["isMultiWeek"]],
                 "C": [estimators[estimatorId]["C"]]
                }
        else:
            raise ValueError('Tried to save results for an unsupported estimator')
        return pd.DataFrame(data=res)

        
    
    

# training
for iteration in range(0,30):
    # we just train one model per task
    task = estimators[estimatorId]["task"]

    # get train labels for all weeks
    y_train = train_labels[["output_"+task+"_"+str(outputWeekNumber) for outputWeekNumber in range(0,numberOfOutputWeeks)]].values
    
    # get validation labels for all weeks (used for early stopping)
    y_valid1And2 = validation1And2_labels[["output_"+task+"_"+str(outputWeekNumber) for outputWeekNumber in range(0,numberOfOutputWeeks)]].values 
    # construct Keras model
    estimator = genericKerasModel(
        estimators[estimatorId]["learningRate"],
        estimators[estimatorId]["hiddenLayers"],
        estimators[estimatorId]["dropoutValue"],
        estimators[estimatorId]["l1reg"],
        estimators[estimatorId]["alpha"],
        estimators[estimatorId]["isMultiWeek"]
    )
    # fit Keras model
    history = estimator.fit(X_train, 
                          y_train, 
                          batch_size=32, 
                          epochs=1000, 
                          verbose=0, 
                          validation_data=(X_valid1And2,y_valid1And2), 
                          callbacks=[keras.callbacks.EarlyStopping(patience=10)])
    '''
    if not os.path.exists('models/'):
        os.makedirs('models')
    estimator.save("models/"+str(estimatorId)+"_"+task)
    '''
    numberOfRanEpochs = len(history.history['loss'])
    temp = constructResults(estimator, task, -1, numberOfRanEpochs)
    temp['iteration'] = iteration
    results = results.append(temp, ignore_index = True)

        
    
def generateModelId(dictionary):
    modelId = ""
    for key in dictionary.keys():
        modelId = modelId + key +"="+ str(dictionary[key]) +"/"
    modelId = modelId[0:-1]
    return modelId

# add a modelId
results["modelId"] = generateModelId(estimators[estimatorId])
results["modelIdNumber"] = estimatorId
                
if not os.path.exists('results_selection/'):
    os.makedirs('results_selection')
results.to_csv("results_selection/"+str(selectedEstimator)+".csv", header=True, index=False)